In [6]:
#@title Basic Libaries { run: "auto" }
#@markdown Getting started with the essential libraries

import os
import sys
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
sb.set(style="darkgrid", context="talk")
plt.style.use("dark_background")
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})

# Add-on preferences
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings as wn
wn.simplefilter(action="ignore", category=FutureWarning)

from copy import deepcopy

In [7]:
#@title Global Attributes { run: "auto" }
#@markdown Define environment states

onDrive = False #@param {type:"boolean"}

try:
  import google.colab
  onColab = True

  from google.colab import files
  
  if onDrive:
    from google.colab import drive
    drive.mount('drive')
except ImportError or ModuleNotFoundError:
  onColab = False
  onDrive = False

print(f"Using Google Colab: {onColab}");
print(f"Using Google Drive: {onDrive}");

Using Google Colab: True
Using Google Drive: False


## Cryptocurrency

---

In [8]:
#@title Add-on Libaries { run: "auto" }

if onColab:
  !{sys.executable} -m pip install -q yfinance

import yfinance as yf
import math
from datetime import datetime
from typing import List

     |████████████████████████████████| 6.4 MB 10.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [9]:
#@title Data Retrieval { run: "auto" }
#@markdown Fetch ticker historical data from YahooFinance

coinSymbols = 'BTC' #@param ["BTC", "ETH"]{allow-input: true}

# List crypto tickers
coins = [coinSymbols]
coinTickers = [f"{c}-USD" for c in coins]

# Fetch all crypto tickers
coinTickerData = yf.download(tickers = " ".join(coinTickers), period="max", group_by = 'ticker', threads = True, auto_adjust = True, prepost = True).reset_index()
coinTickerData.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Volume
2768,2022-04-16,"40,552.32","40,633.68","40,078.43","40,424.48",16833150693
2769,2022-04-17,"40,417.78","40,570.73","39,620.89","39,716.95",19087633042
2770,2022-04-18,"39,721.20","40,986.32","38,696.19","40,826.21",33705182072
2771,2022-04-19,"40,828.18","41,672.96","40,618.63","41,502.75",25303206547
2772,2022-04-20,"41,453.36","42,091.90","41,282.08","42,068.30",27645448192


In [10]:
#@title Catalogue and Format Dataset  { run: "auto" }
#@markdown Stores all ticker dataset into single repo

tickerData = deepcopy(coinTickerData)
allTickers = {}

for idx, ticker in enumerate(coinTickers):
  # Retrieve main ticker information
  tickerInfo = yf.Ticker(ticker).info
  tName = tickerInfo["name"]
  tNameShort = tickerInfo["fromCurrency"]

  # Extract focal data
  if(len(coinTickers) > 1):
    data = tickerData[["Date" , ticker]]

    # Flatten the dataframe headers
    if(len(data) > 1):
      data.columns = data.columns.get_level_values(1)
      data.columns.values[0] = "Date"
  else:
    data = tickerData

  # Format headers
  data.columns = data.columns.str.upper()

  # Store into data repo
  allTickers[ticker] = {
    "info": {
        "name": tName,
        "shortName": tNameShort
    },
    "data": data
  }

# Display brief info about tickers
pd.DataFrame.from_dict(allTickers, orient='index').iloc[:, :-1]

,info
BTC-USD,"{'name': 'Bitcoin', 'shortName': 'BTC'}"


In [11]:
#@title Extract Specific Tickers Columns  { run: "auto" }
#@markdown Stores all ticker dataset into single repo

def getTickerSeries(src:pd.DataFrame, ticker:str, cols:List[str] = []) -> pd.DataFrame:
  tickerData = deepcopy(src[ticker]["data"])
  if(len(cols) > 0) and set(cols).issubset(set(tickerData.columns.to_list())):
    tickerData = tickerData[cols]
  return tickerData

# targetCols=["DATE", "CLOSE"]
targetCols = '' #@param {type:"string"}
cols = list(map(str.strip, targetCols.split(',')))

# Initialise with first ticker
tickerClose = getTickerSeries(src=allTickers, ticker=coinTickers[0], cols=cols)

# Append if there are more tickers
if len(coinTickers) > 1:  
  for ticker in coinTickers[1:]:
    tickerClose = pd.merge(tickerClose, getTickerSeries(src=allTickers, ticker=ticker, cols=cols), on="DATE", suffixes=tuple(f"_{coin}" for coin in coins))

# Display data preview
tickerClose = tickerClose.set_index('DATE')
tickerClose.tail()

,OPEN,HIGH,LOW,CLOSE,VOLUME
DATE,,,,,
2022-04-16,"40,552.32","40,633.68","40,078.43","40,424.48",16833150693
2022-04-17,"40,417.78","40,570.73","39,620.89","39,716.95",19087633042
2022-04-18,"39,721.20","40,986.32","38,696.19","40,826.21",33705182072
2022-04-19,"40,828.18","41,672.96","40,618.63","41,502.75",25303206547
2022-04-20,"41,453.36","42,091.90","41,282.08","42,068.30",27645448192


## Google Trends (Annual Archive)

----


In [12]:
#@title Add-on Libaries { run: "auto" }

if onColab:
  !{sys.executable} -m pip install -q pytrends iteration_utilities

from pytrends.request import TrendReq
from datetime import date
from typing import List
from iteration_utilities import grouper
import glob

     |████████████████████████████████| 283 kB 5.9 MB/s 


In [ ]:
#@title Scrape Trend Data 
#@markdown Archived search terms, with specified timeframe

terms = 'cryptocurrency' #@param {type:"string"}
if len(terms) == 0:
  terms = 'cryptocurrency'
searchTerms = [t for t in list(map(str.strip, terms.split(','))) if len(t) > 0]


minYear =  2021 #@param {type:"number"}
minMth, minDay = 1,1
maxYear = date.today().year
maxMth, maxDay = 12,31

if minYear < maxYear - 5:
  print(f"{minYear} is too far back, min. set to start from {maxYear - 5}")
  minYear = maxYear - 5
singleYear = True #@param {type:"boolean"}

# Retrive archive data for a specific year
def annualArchive(searchTerms:List[str], year:int):
  googleTrend = TrendReq()
  googleTrend.build_payload(kw_list=searchTerms, timeframe="all")
  searchTrendData = googleTrend.get_historical_interest(searchTerms, year_start=year, month_start=minMth, day_start=minDay, hour_start=0, year_end=year, month_end=maxMth, day_end=maxDay, hour_end=0, cat=0, geo='', gprop='', sleep=0)
  searchTrendData.reset_index(inplace=True)
  searchTrendData.columns.values[0] = "dtime"
  searchTrendData.set_index('dtime', inplace=True)
  return searchTrendData

# Centralised archive repo
trendArchive = {}

if singleYear:
  # Append to archive repo
  trendArchive[minYear] = annualArchive(searchTerms, minYear)
else:
  for year in range(minYear, maxYear):
    # Append to archive repo
    trendArchive[minYear] = annualArchive(searchTerms, year)

# Merge all into single archive dataset
trendArchiveData = pd.concat(list(trendArchive.values()))
trendArchiveData

#@markdown ---
#@markdown Export Dataset<br/>

exportCSV = True #@param {type:"boolean"}

try:
  if len(trendArchiveData) > 1 and exportCSV:

    # Export configurations
    fileName = f"trend-{minYear}.csv" if singleYear else f"trend-{minYear}_{maxYear-1}.csv"
    exportFolderPath = "" #@param {type:"string"}
    if len(exportFolderPath) > 0 and not exportFolderPath.endswith('/'):
      exportFolderPath += '/'
    filePath = f'{exportFolderPath if len(exportFolderPath) > 0 else ""}{fileName}'

    # Export destinations
    if onColab:
      if onDrive:
        filePath = f'/content/drive/My Drive/{filePath}'
        trendArchiveData.to_csv(filePath, encoding='utf-8', index=False)
        print(f"SUCCESSFULL EXPORTED to [{filePath}]")
      else:
        trendArchiveData.to_csv(fileName) 
        files.download(fileName)
        print(f"SUCCESSFULL EXPORTED to [{fileName}]")
    else:
      BASE_DIR = os.getcwd()
      filePath = os.path.join(BASE_DIR, filePath)
      trendArchiveData.to_csv(filePath, encoding='utf-8', index=False)
      print(f"SUCCESSFULL EXPORTED to [{filePath}]")

except NameError:
  print("NO DATA TO EXPORT")

,cryptocurrency,isPartial
dtime,,
2021-01-01 00:00:00,23,False
2021-01-01 01:00:00,27,False
2021-01-01 02:00:00,40,False
2021-01-01 03:00:00,27,False
2021-01-01 04:00:00,41,False
...,...,...
2021-12-30 21:00:00,46,False
2021-12-30 22:00:00,52,False
2021-12-30 23:00:00,57,False


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESSFULL EXPORTED to [trend-2021.csv]


### Compilation Merge

----


In [ ]:
combineCSV = True #@param {type:"boolean"}

if combineCSV:
  # Import source
  importFolderPath = "" #@param {type:"string"}
  if len(importFolderPath) > 0 and not importFolderPath.endswith('/'):
    importFolderPath += '/'
  if onColab:
    if onDrive:
      importFolderPath = f'/content/drive/My Drive/{importFolderPath if len(importFolderPath) > 0 else ""}'
    else:
      importFolderPath = f"/content/{importFolderPath if len(importFolderPath) > 0 else ''}"
  filesPath = f'{importFolderPath}*.csv'

  # Import process
  allTrends = []
  try:
    if len(glob.glob(filesPath)):
      for f in glob.glob(filesPath):
        trendData = pd.read_csv(f, header=0).dropna()
        trendData
        allTrends.append(trendData)

      if len(allTrends) == len(glob.glob(filesPath)):
        print(f"SUCCESSFULLY IMPORTED from [{filesPath}]")
        allTrends = pd.concat(allTrends, axis=1)
        _, i = np.unique(allTrends.columns, return_index=True)
        allTrends = allTrends.iloc[:, i]
        allTrends['dtime'] = pd.to_datetime(allTrends['dtime'], errors='coerce')
        # allTrends.set_index('dtime', inplace=True)
        allTrends
        
        # @markdown ---
        # @markdown Export Dataset<br/>

        exportCSV = True #@param {type:"boolean"}

        if exportCSV:
          #  Retrieve Historical Time Range from Data
          earliestDate = allTrends.head(1)['dtime'].values[0].astype('datetime64[D]').item()
          latestDate = allTrends.tail(1)['dtime'].values[0].astype('datetime64[D]').item()
          print(f"Historical year range: {earliestDate.year} to {latestDate.year}")

          # Export configurations
          exportFolderPath = "" #@param {type:"string"} 
          fileName = f"trend-{earliestDate.year}.csv" if earliestDate.year == latestDate.year else f"trend-{earliestDate.year}_{latestDate.year}.csv"
          if len(exportFolderPath) > 0 and not exportFolderPath.endswith('/'):
            exportFolderPath += '/'
          filePath = f'{exportFolderPath if len(exportFolderPath) > 0 else ""}{fileName}'

          # Export destinations
          if onColab:
            if onDrive:
              filePath = f'/content/drive/My Drive/{filePath}'
              allTrends.to_csv(filePath, encoding='utf-8', index=False)
              print(f"SUCCESSFULL EXPORTED to [{filePath}]")
            else:
              allTrends.to_csv(fileName, encoding='utf-8', index=False)
              files.download(fileName)
              print(f"SUCCESSFULL EXPORTED to [{fileName}]")
          else:
            BASE_DIR = os.getcwd()
            filePath = os.path.join(BASE_DIR, filePath)
            allTrends.to_csv(filePath, encoding='utf-8', index=False)
            print(f"SUCCESSFULL EXPORTED to [{filePath}]")

  except NameError:
    print("NO DATA TO IMPORT")
  except FileNotFoundError:
      print("ERROR - FILE NOT FOUND. [Run 'Trends Scraper' to export the file.]")


,dtime,cryptocurrency,isPartial
0,2017-01-01 00:00:00,30,False
1,2017-01-01 01:00:00,16,False
2,2017-01-01 02:00:00,17,False
3,2017-01-01 03:00:00,54,False
4,2017-01-01 04:00:00,18,False
...,...,...,...
43968,2021-12-30 21:00:00,46,False
43969,2021-12-30 22:00:00,52,False
43970,2021-12-30 23:00:00,57,False
43971,2021-12-31 00:00:00,74,False


,dtime,bitcoin,isPartial
0,2017-01-01 00:00:00,28,False
1,2017-01-01 01:00:00,32,False
2,2017-01-01 02:00:00,33,False
3,2017-01-01 03:00:00,41,False
4,2017-01-01 04:00:00,33,False
...,...,...,...
43968,2021-12-30 21:00:00,80,False
43969,2021-12-30 22:00:00,85,False
43970,2021-12-30 23:00:00,77,False
43971,2021-12-31 00:00:00,70,False


SUCCESSFULLY IMPORTED from [/content/*.csv]


,bitcoin,cryptocurrency,dtime,isPartial
0,28,30,2017-01-01 00:00:00,False
1,32,16,2017-01-01 01:00:00,False
2,33,17,2017-01-01 02:00:00,False
3,41,54,2017-01-01 03:00:00,False
4,33,18,2017-01-01 04:00:00,False
...,...,...,...,...
43968,80,46,2021-12-30 21:00:00,False
43969,85,52,2021-12-30 22:00:00,False
43970,77,57,2021-12-30 23:00:00,False
43971,70,74,2021-12-31 00:00:00,False


Historical year range: 2017 to 2021


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESSFULL EXPORTED to [trend-2017_2021.csv]


## Dataset Merger (Crypto + Trends)

---

In [13]:
#@title Partial Trends Scraper { run: "auto" }
#@markdown Fill in the archive data with latest data to match the cryptocurrency dataset

# Retrieve Time Range from TickerData
minYear = (coinTickerData.head(1)['Date'].values[0].astype('datetime64[D]').item()).year
latestDate = (coinTickerData.tail(1)['Date'].values[0].astype('datetime64[D]').item())
maxYear = latestDate.year
if minYear < maxYear - 5:
  print(f"{minYear} is too far back, min. set to start from {maxYear - 5}")
  minYear = maxYear - 5
print(f"Year Range: {minYear} to {maxYear -1}")

fileName = f"trend-{minYear}.csv" if minYear == maxYear-1 else f"trend-{minYear}_{maxYear-1}.csv"
folderPath = "" #@param {type:"string"}
if len(folderPath) > 0 and not folderPath.endswith('/'):
  folderPath += '/'

if onColab:
  if onDrive:
    folderPath = f'/content/drive/My Drive/{folderPath if len(folderPath) > 0 else ""}'
  else:
    folderPath = f"/content/{folderPath if len(folderPath) > 0 else ''}"
else:
  BASE_DIR = os.getcwd()
  folderPath = os.path.join(BASE_DIR, f"{folderPath if len(folderPath) > 0 else ''}")
filePath = f'{folderPath}{fileName}'

# Import CSV 
try:
  allTrendData = pd.read_csv(filePath, header=0)
  allTrendData.dropna(inplace=True)

  # Retrive search terms from dataset
  searchTerms = [term for term in list(set(allTrendData.columns) - set(['dtime', 'isPartial']))]

  # Reassign dtypes
  allTrendData['dtime'] = pd.to_datetime(allTrendData['dtime'], errors='coerce')
  for idx, val in enumerate(searchTerms):
    allTrendData[searchTerms[idx]] = pd.to_numeric(allTrendData[searchTerms[idx]], errors='coerce')
  allTrendData['isPartial'] = allTrendData['isPartial'].astype('bool')

  # Retrieve current trend data
  recentTrendData = []
  if len(searchTerms) > 0:
    for term in searchTerms:
      googleTrend = TrendReq()
      googleTrend.build_payload(kw_list=[term], timeframe="all")
      searchTrendData = googleTrend.get_historical_interest([term], year_start=latestDate.year, month_start=1, day_start=1, hour_start=0, year_end=latestDate.year, month_end=latestDate.month, day_end=latestDate.day, hour_end=int(datetime.now().strftime("%H")), cat=0, geo='', gprop='', sleep=0)
      searchTrendData.reset_index(inplace=True)
      searchTrendData.columns.values[0] = "dtime"

      recentTrendData.append(searchTrendData)
      
  recentTrendData = pd.concat(recentTrendData, axis=1)
  _, i = np.unique(recentTrendData.columns, return_index=True)
  recentTrendData = recentTrendData.iloc[:, i]
  recentTrendData['dtime'] = pd.to_datetime(recentTrendData['dtime'], errors='coerce')

  # Update search trend dataset with recent
  allTrendData = allTrendData.merge(recentTrendData, how="outer")

  allTrendData
except FileNotFoundError:
    print("ERROR - FILE NOT FOUND. [Run 'Trends Scraper' to export the file.]")

2014 is too far back, min. set to start from 2017
Year Range: 2017 to 2021


,bitcoin,cryptocurrency,dtime,isPartial
0,28,30,2017-01-01 00:00:00,True
1,32,16,2017-01-01 01:00:00,True
2,33,17,2017-01-01 02:00:00,True
3,41,54,2017-01-01 03:00:00,True
4,33,18,2017-01-01 04:00:00,True
...,...,...,...,...
46608,74,49,2022-04-20 08:00:00,False
46609,75,46,2022-04-20 09:00:00,False
46610,79,56,2022-04-20 10:00:00,False
46611,80,46,2022-04-20 11:00:00,False


### Timeframe Specific

---

In [14]:
#@title Hourly Search Trends { run: "auto" }
#@markdown View the hourly trend

searchTrend = deepcopy(allTrendData)

# Dataframe formatting
if "dtime" in searchTrend.columns:
  searchTrend['date'] = pd.to_datetime(searchTrend['dtime']).dt.date
  searchTrend['time'] = pd.to_datetime(searchTrend['dtime']).dt.time
  searchTrend.drop(['dtime', 'isPartial'], axis=1, inplace=True)
searchTrend.columns = searchTrend.columns.str.upper()

# Preview data
searchTrend

,BITCOIN,CRYPTOCURRENCY,DATE,TIME
0,28,30,2017-01-01,00:00:00
1,32,16,2017-01-01,01:00:00
2,33,17,2017-01-01,02:00:00
3,41,54,2017-01-01,03:00:00
4,33,18,2017-01-01,04:00:00
...,...,...,...,...
46608,74,49,2022-04-20,08:00:00
46609,75,46,2022-04-20,09:00:00
46610,79,56,2022-04-20,10:00:00
46611,80,46,2022-04-20,11:00:00


**Daily Average Search Percentile**

In [15]:
#@title Daily Average Search Percentile { run: "auto" }
#@markdown The mean interest per day

avgDailySearchTrend = deepcopy(searchTrend)

# Dataframe formatting
avgDailySearchTrend = avgDailySearchTrend.groupby(['Date'.upper()]).mean().round(0)
avgDailySearchTrend.reset_index(inplace=True)
avgDailySearchTrend['Date'.upper()] = avgDailySearchTrend['Date'.upper()].astype('datetime64[ns]')

# Preview data
avgDailySearchTrend

,DATE,BITCOIN,CRYPTOCURRENCY
0,2017-01-01,37.00,18.00
1,2017-01-02,61.00,27.00
2,2017-01-03,59.00,34.00
3,2017-01-04,66.00,32.00
4,2017-01-05,82.00,34.00
...,...,...,...
1931,2022-04-16,72.00,54.00
1932,2022-04-17,72.00,54.00
1933,2022-04-18,87.00,65.00
1934,2022-04-19,82.00,55.00


In [16]:
#@title Monthly Search Percentile { run: "auto" }
#@markdown Rescrape monthly search trend

# Scrape from Google Trends 
googleTrend.build_payload(kw_list=searchTerms, timeframe="all")
monthlySearchTrend = googleTrend.interest_over_time()
monthlySearchTrend.reset_index(inplace=True)

# Dataframe formatting
monthlySearchTrend.drop(['isPartial'], axis=1, inplace=True)
monthlySearchTrend.columns = monthlySearchTrend.columns.str.upper()

# Preview data
monthlySearchTrend

,DATE,CRYPTOCURRENCY,BITCOIN
0,2004-01-01,0,0
1,2004-02-01,0,0
2,2004-03-01,0,0
3,2004-04-01,0,0
4,2004-05-01,0,0
...,...,...,...
215,2021-12-01,4,35
216,2022-01-01,4,38
217,2022-02-01,3,33
218,2022-03-01,2,29


In [17]:
#@title Crypto - Trends Combined Dataset  { run: "auto" }
#@markdown Merge / Export the above datasets

exportCSV = True #@param {type:"boolean"}

# Prep Crypto dataset
closes = deepcopy(tickerClose)
# Prep Trends dataset
trend = deepcopy(avgDailySearchTrend)

# Merge with corresponding trend data
closeTrend = pd.merge(closes, trend, on="Date".upper())
closeTrend.dropna(inplace=True)
# Data formatting
closeTrend.columns = closeTrend.columns.str.upper()
# Display dataset preview
closeTrend
closeTrend.shape

# Export to CSV
if exportCSV:
  fileName = f"{coins[0]}-SearchTrend.csv"
  folderPath = "" #@param {type:"string"} 
  if len(folderPath) > 0 and not folderPath.endswith('/'):
    folderPath += '/'
  filePath = f'{folderPath if len(folderPath) > 0 else ""}{fileName}'

  if onColab:
    if onDrive:
      filePath = f'/content/drive/My Drive/{filePath}'
      closeTrend.to_csv(filePath, encoding='utf-8', index=False)
      print(f"SUCCESSFULL EXPORTED to [{filePath}]")
    else:
      closeTrend.to_csv(fileName, encoding='utf-8', index=False)
      files.download(fileName)
      print(f"SUCCESSFULL EXPORTED to [{fileName}]")
  else:
    BASE_DIR = os.getcwd()
    filePath = os.path.join(BASE_DIR, filePath)
    closeTrend.to_csv(filePath, encoding='utf-8', index=False)
    print(f"SUCCESSFULL EXPORTED to [{filePath}]")

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,BITCOIN,CRYPTOCURRENCY
0,2017-01-01,963.66,"1,003.08",958.70,998.33,147775008,37.00,18.00
1,2017-01-02,998.62,"1,031.39",996.70,"1,021.75",222184992,61.00,27.00
2,2017-01-03,"1,021.60","1,044.08","1,021.60","1,043.84",185168000,59.00,34.00
3,2017-01-04,"1,044.40","1,159.42","1,044.40","1,154.73",344945984,66.00,32.00
4,2017-01-05,"1,156.73","1,191.10",910.42,"1,013.38",510199008,82.00,34.00
...,...,...,...,...,...,...,...,...
1931,2022-04-16,"40,552.32","40,633.68","40,078.43","40,424.48",16833150693,72.00,54.00
1932,2022-04-17,"40,417.78","40,570.73","39,620.89","39,716.95",19087633042,72.00,54.00
1933,2022-04-18,"39,721.20","40,986.32","38,696.19","40,826.21",33705182072,87.00,65.00
1934,2022-04-19,"40,828.18","41,672.96","40,618.63","41,502.75",25303206547,82.00,55.00


(1936, 8)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SUCCESSFULL EXPORTED to [BTC-SearchTrend.csv]


---